In [2]:
import os
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    trained_model_path:Path
    updated_base_model_path:Path
    training_data:Path
    params_epoch: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [4]:
from src.ChickenDisease.constants import *
from src.ChickenDisease.utils.common import read_yaml, create_directories
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split


In [5]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAM_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = self.config.data_ingestion
        create_directories([ Path(training.root_dir)])


        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            training_data=Path(training_data.local_data_file),
            updated_base_model_path= Path(prepare_base_model.updated_model_path),
            params_epoch=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        
        return training_config

In [5]:
import time

In [6]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.trained_model_path
        )

    def data_split(self):
        df=pd.read_csv(self.config.training_data)
        trsplit=.9
        vsplit=.05
        dsplit=vsplit/(1-trsplit)

        strat=df['labels']
        self.train_df, dum_df= train_test_split(df, train_size= 0.9, shuffle=True, random_state=123, stratify=strat)

        strat=dum_df['labels']
        self.test_df,self.valid_df=train_test_split(dum_df,train_size=.9, shuffle=True, random_state=123, stratify=strat)
        print(f'train_df length:{len(self.train_df)}, test_df length:{len(self.test_df)},  valid_df length:{len(self.valid_df)}')

        
    
    def trim(self, max_size=500, min_size=0, column="labels"):
            df=self.train_df.copy() #copy the dataframe
            original_class_count= len(list(df['labels'].unique()))
            print("original class count is:", original_class_count)
            sample_list=[]
            groups= df.groupby('labels')

            for label in df[column].unique():
                group=groups.get_group(label)
                sample_count= len(group)
                if sample_count> max_size:
                    strat= group[column]
                    samples,_= train_test_split(group, train_size= max_size, shuffle=True, random_state=123, stratify= strat)
                    sample_list.append(samples)
                elif sample_count>=min_size:
                    sample_list.append(group)
            
            df=pd.concat(sample_list, axis=0).reset_index(drop=True)

            final_class_count=len(list(df[column].unique()))
            if final_class_count != original_class_count:
                print("***WARNING*** Data frame has been reduced")
            
            balance= list(df[column].value_counts())
            print(balance)
            return df


    def train_valid_generator(self):
        img_size= self.config.params_image_size
        batch_size= self.config.params_batch_size 
            
        trgen=tf.keras.preprocessing.image.ImageDataGenerator( rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest' )

        t_and_v_gen=tf.keras.preprocessing.image.ImageDataGenerator()

        self.train_gen=trgen.flow_from_dataframe(self.train_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                        class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)
        self.valid_gen=t_and_v_gen.flow_from_dataframe(self.valid_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                        class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=batch_size)
        # for the test_gen we want to calculate the batch size and test steps such that batch_size X test_steps= number of samples in test set
        # this insures that we go through all the sample in the test set exactly once.
        length=len(self.test_df)
        test_batch_size=sorted([int(length/n) for n in range(1,length+1) if length % n ==0 and length/n<=80],reverse=True)[0]  
        test_steps=int(length/test_batch_size)
        self.test_gen=t_and_v_gen.flow_from_dataframe(self.test_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                        class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=test_batch_size)
        # from the generator we can get information we will need later
        classes=list(self.train_gen.class_indices.keys())
        class_indices=list(self.train_gen.class_indices.values())
        class_count=len(classes)
        labels=self.test_gen.labels
        print ( 'test batch size: ' ,test_batch_size, '  test steps: ', test_steps, ' number of classes: ', class_count)
        print ('{0:^25s}{1:^12s}'.format('class name', 'class index'))
        for klass, index in zip(classes, class_indices):
            print(f'{klass:^25s}{str(index):^12s}')
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_gen.samples // self.train_gen.batch_size
        self.validation_steps = self.valid_gen.samples // self.valid_gen.batch_size

        self.model.fit(
            self.train_gen,
            epochs=self.config.params_epoch,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_gen,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [1]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.data_split()
    training.trim()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

NameError: name 'ConfigurationManager' is not defined